In [16]:
from py2neo import Graph
from py2neo.bulk import create_nodes, create_relationships
import pandas as pd
import json
!pip install monotonic


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
py2neo 2021.2.3 requires pansi>=2020.7.3, which is not installed.


In [7]:
#load csv as dataframe
im_data = pd.read_csv("sf_dataset.csv")
   

In [8]:
#im_data["deviceID"] = im_data["deviceID"].astype(str)

# create dataframes for node
df_business = im_data.filter(["business_id","business_name","business_address","latitude","longitude"])
# drop the duplicates so that we just try to insert the node once (multiple times will not work)
df_business = df_business.drop_duplicates('business_id', keep='last')
# turn the dataframe into json string
json_business = df_business.to_json(orient="records")
# trun json into dictionarie for the import
dict_business = json.loads(json_business)

In [12]:
# prepare Zip node data
df_zip = im_data.filter(["zip"])
df_zip = df_zip.drop_duplicates('zip', keep='last')
json_zip = df_zip.to_json(orient="records")
dict_zip = json.loads(json_zip)

# prepare person data
df_person = im_data.filter(["user_name","deviceID"])
df_person = df_person.drop_duplicates('deviceID', keep='last')
print(len(df_person))
json_person = df_person.to_json(orient="records")
dict_person = json.loads(json_person)

# create dataframes for relationships
df_relationship = im_data.filter(["business_id","deviceID","scan_timestamp"])
json_relationship = df_relationship.to_json(orient="records")
dict_relationship = json.loads(json_relationship)

df_relationship_zip = im_data.filter(["business_id","zip"])
df_relationship_zip = df_relationship_zip.drop_duplicates('business_id', keep='last')
json_relationship_zip = df_relationship_zip.to_json(orient="records")
dict_relationship_zip = json.loads(json_relationship_zip)


#print(dict_relationship_zip)
#print(dict_relationship)


4998


In [17]:
#connect to neo4j
graph = Graph("neo4j+s://d684082d.databases.neo4j.io", auth=("neo4j", "0IvK23_mmzvdXeR2KZHA2PqtEw029zTevtQQaePe_tc"))

In [18]:
# Create business nodes
create_nodes(graph.auto(), dict_business, labels={"Business"})
print(graph.nodes.match("Business").count())

# Create zip nodes
create_nodes(graph.auto(), dict_zip, labels={"Zip"})
print(graph.nodes.match("Zip").count())

# create person nodes
create_nodes(graph.auto(), dict_person, labels={"Person"})
print(graph.nodes.match("Person").count())


#create relationships people visiting businesses
ex_people = []

for p in dict_relationship:
    
    device= p["deviceID"]
    business = p["business_id"]

    p.pop("deviceID")
    p.pop("business_id")
    ex_people.append((device,p,business))

#print(ex_people)

#man_relationship = [("5403628525158",{"scan_timestamp":"2022-03-05 23:19:25"},"0311287-01-001")]
rel_pb = create_relationships(graph.auto(), ex_people, "VISITED", start_node_key=("Person", "deviceID"), end_node_key=("Business", "business_id"))
print(rel_pb)

#create zip and business
ex_zip = []

for p in dict_relationship_zip:

    zip= p["zip"]
    business = p["business_id"]
    ex_zip.append((business,{"active":1},zip))

#print(ex_zip)

rel_bz = create_relationships(graph.auto(), ex_zip, "ISLOCATED", start_node_key=("Business", "business_id"), end_node_key=("Zip", "zip"))
print(rel_bz)



2500
36
4998
None
None


In [22]:

from py2neo import Graph, Node, Relationship, NodeMatcher
from sqlalchemy import false, null


g = Graph("neo4j+s://d684082d.databases.neo4j.io", auth=("neo4j", "0IvK23_mmzvdXeR2KZHA2PqtEw029zTevtQQaePe_tc"))
matcher = NodeMatcher(g)

# Start the transaction
tx = g.begin()
try:
    #Create the user node (does not check of node already exists!!)
    a = matcher.match("Person", device_id="00000000000039").first()
    if (a== None):

        a = Node("Person", device_id="00000000000039", user_name = "Likith Vinayaka")
        tx.create(a)

        # check if node has been created if not raise exception to rollback transaction
        if(tx.exists(a) == False):
            raise (Exception) 

    # create the relationship (user)-[VISITED]-(Business)
    b = matcher.match("Business", business_id="0322120-04-001").first()

    propierties = {"scan_timestamp":"2022-01-01 12:55:55"}
    r = Relationship(a, "VISITED", b, **propierties)
    r.identity = None
    tx.create(r)
    
    # check if relationship has been created if not raise exception to rollback transaction
    if(tx.exists(r) == False):
        raise (Exception)

    # commit the transaction if everything was successful
    g.commit(tx)

except Exception as e:
    g.rollback(tx) # rollback the transaction on error
    print(e)




"""
For querying in neo4j browser

MATCH (p:Person)
WHERE p.user_name = 'Likith Vinayaka'
RETURN p
   
MATCH (b:Business)
WHERE b.business_id = '0322120-04-001'
RETURN b   

"""

"\nUse queries in neo4j browser\n\nMATCH (p:Person)\nWHERE p.user_name = 'Likith Vinayaka'\nRETURN p\n   \nMATCH (b:Business)\nWHERE b.business_id = '0322120-04-001'\nRETURN b   \n\n"

In [23]:
from py2neo import Graph

import pprint

# create connection
g = Graph("neo4j+s://d684082d.databases.neo4j.io", auth=("neo4j", "0IvK23_mmzvdXeR2KZHA2PqtEw029zTevtQQaePe_tc"))

# [Example 1]

# Find where Gilbert went and which zip code the business is in
# Using :ISLOCATED only shows the zip (directional + relation specified)
result = g.run("MATCH (p:Person)-[VISITED]-(Business)-[:ISLOCATED]-(Zip)\
                WHERE p.user_name = 'Gilbert Harris'\
                RETURN p,Business,Zip").data()
#pprint.pprint(result)
pprint.pprint(len(result))

### Same thing just with an arrow instead of naming the relationship
result = g.run("MATCH (p:Person)-->(Business)-->(Zip)\
                WHERE p.user_name = 'Gilbert Harris'\
                RETURN p,Business,Zip").data()
#pprint.pprint(result)
pprint.pprint(len(result))

### Matches more, because the zip doesnt have a direction
# Go to AuraDB query UI
result = g.run("MATCH (p:Person)--(Business)--(Zip)\
                WHERE p.user_name = 'Gilbert Harris'\
                RETURN p,Business,Zip").data()
#pprint.pprint(result)
pprint.pprint(len(result))



# [Example 2]
### Find everyone who went to a Bank using a regex
### Play around with the relation and see different results try (-- or <-- or --> ) and expore the results
result = g.run("MATCH (Business)-[vis:VISITED]-(Person)\
                WHERE Business.business_name =~ '.*Bank.*'\
                RETURN Business,Person").data()
#pprint.pprint(result)
pprint.pprint(len(result))

### Find everyone who visited a bank on Jan 3rd
result = g.run("MATCH (p:Person)-[vis:VISITED]-(b:Business)\
                WHERE vis.scan_timestamp =~ '2022-01-03.*' \
                AND b.business_name =~ '.*Bank.*'\
                RETURN b, p").data()
#pprint.pprint(result)
pprint.pprint(len(result))


8
8
160
140
49
